# 📓 02 - Introdução ao FAISS e buscas vetoriais

Neste notebook, vamos explorar como:
✅ Gerar embeddings de texto.  
✅ Indexar esses embeddings no FAISS.  
✅ Fazer buscas vetoriais simples e entender similaridade.

---

## ⚙️ Setup inicial

Vamos começar instalando as bibliotecas necessárias.

```python
# Instale as bibliotecas se ainda não estiverem
# !pip install faiss-cpu sentence-transformers

import faiss
from sentence_transformers import SentenceTransformer
import numpy as np
```

In [2]:
import faiss
from sentence_transformers import SentenceTransformer
import numpy as np

c:\Users\vinicius\Documents\Projetos\RAG-study\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 📝 1️⃣ Gerar embeddings de texto
Vamos usar o modelo all-MiniLM-L6-v2 do Sentence Transformers, leve e eficiente para testes iniciais.

In [3]:
# Carregar modelo de embeddings
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# Lista de textos de exemplo
texts = [
    "Retrieval-Augmented Generation (RAG) combina recuperação de informações e geração de linguagem natural.",
    "O sol nasce no leste e se põe no oeste.",
    "Python é uma linguagem de programação muito popular.",
    "Modelos de linguagem podem gerar respostas criativas.",
    "A IA está revolucionando diversos setores."
]

# Gerar embeddings (convertidos para numpy)
embeddings = model.encode(texts, convert_to_numpy=True)
print("🔹 Embeddings shape:", embeddings.shape)

c:\Users\vinicius\Documents\Projetos\RAG-study\.venv\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\vinicius\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not inst

🔹 Embeddings shape: (5, 384)


## 🔍 2️⃣ Indexar embeddings no FAISS

In [4]:
# Dimensão dos embeddings (deve bater com a saída do modelo)
dimension = embeddings.shape[1]

# Inicializar o índice FAISS (IndexFlatIP = Inner Product / similaridade por produto interno)
index = faiss.IndexFlatIP(dimension)

# Normalizar embeddings para usar similaridade de cosseno
faiss.normalize_L2(embeddings)

# Adicionar ao índice
index.add(embeddings)
print("🔹 Número de vetores indexados:", index.ntotal)

🔹 Número de vetores indexados: 5


## 🔎 3️⃣ Fazer uma busca vetorial simples

In [5]:
# Pergunta de exemplo
query = "Como a IA está mudando as indústrias?"

# Gerar embedding da pergunta
query_embedding = model.encode([query], convert_to_numpy=True)
faiss.normalize_L2(query_embedding)

# Buscar top-3 resultados mais similares
k = 3
distances, indices = index.search(query_embedding, k)

print("🔹 Resultados:")
for i, idx in enumerate(indices[0]):
    print(f"Rank {i+1}: Texto: {texts[idx]} (Score: {distances[0][i]:.4f})")

🔹 Resultados:
Rank 1: Texto: A IA está revolucionando diversos setores. (Score: 0.5576)
Rank 2: Texto: Python é uma linguagem de programação muito popular. (Score: 0.3936)
Rank 3: Texto: Modelos de linguagem podem gerar respostas criativas. (Score: 0.3435)
